# Kaggle Compeition with Titanic Dataset using ML.NET AutoML

This notebook shows how to anticipate the [well-known titanic competition](https://www.kaggle.com/c/titanic) on kaggle using ml.net. In this notebook, you will learn how to
- Create pipeline for titanic
- Use AutoML API to run hyper-parameter optimization on that pipeline and get the best model
- Predict using the best model, and save predicting result to csv for submission

## Install NuGet packages for training ML.NET models and plotting:

In [1]:
// using nightly-build
// #i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
// #i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
// #i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.CSharp"
#r "nuget: Microsoft.ML.AutoML"
#r "nuget: Microsoft.Data.Analysis"

Restore sources https://api.nuget.org/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.1 Microsoft.ML.AutoML, 0.21.1 Plotly.NET.CSharp, 0.13.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\scaravelea\.nuget\packages\microsoft.ml.automl\0.21.1\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `C:\Users\scaravelea\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\scaravelea\.nuget\packages\microsoft.data.analysis\0.21.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `C:\Users\scaravelea\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

## Import packages

In [1]:
// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

## Load Dataset
The dataset comes from [kaggle titanic](https://www.kaggle.com/competitions/titanic/data) and is split into __train.csv__ and __test.csv__. __train.csv__ includes feature columns like `Sex`, `Pclass`, etc.. and also ground truth label for us to train a model while __test.csv__ only contains feature columns.

In the following section, we are going to load and split __train.csv__ into training and validation set using `DataFrame` api, and preview the first 10 line of it.

#### Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [1]:
var context = new MLContext();

//Load File
var trainDataPath = EnsureDataSetDownloaded("titanic-train.csv");
var df = DataFrame.LoadCsv(trainDataPath);

var trainTestSplit = context.Data.TrainTestSplit(df, 0.1);
df.Head(10)

### Construct training pipeline
The following code shows how to construct sweepable pipeline with default binary classifiers and search space for hyper-parameter optimization. The sweepable pipeline comes with featurize pipeline first, which transfers columns into a single feature and then feeds into binary classifiers.

In [1]:
var pipeline = context.Auto().Featurizer(df, excludeColumns: new[]{"Survived"})
                        .Append(context.Transforms.Conversion.ConvertType("Survived", "Survived", DataKind.Boolean))
					    .Append(context.Auto().BinaryClassification(labelColumnName: "Survived", useFastForest: false, useSdca: false, useLbfgs: false));

### Run Hyper-parameter optimization using AutoMLExperiment
The following code shows how to use `AutoMLExperiment` to sweep over the sweepable pipeline created before and optimize for the best parameters. During the 300 seconds training time budget, it will train model on `trainTestSplit.TrainSet`, and evaluate model with `trainTestSplit.TestSet` using `Accuracy` metric. While training, the `NotebookMonitor` helps organize the experiment output and plotting it in output panel. 

In [1]:
// Configure AutoML
var monitor = new NotebookMonitor();

var experiment = context.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(60)
                    .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                    .SetEvaluateMetric(BinaryClassificationMetric.Accuracy, "Survived", "PredictedLabel")
                    .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = await experiment.RunAsync();

## Predicting result from __test.csv__ and write it to csv for submission
The following code shows how to consume the best model from previous AutoMLExperiment, generate prediction result and save it to csv. The score for submission should be around 78% and beats over 90% of total submissions.

In [1]:
var bestModel = res.Model;
var testDataPath = EnsureDataSetDownloaded("titanic-test.csv");
var submissionCsvPath = EnsureDataSetDownloaded("titanic-submission.csv");

var testDf = DataFrame.LoadCsv(testDataPath,guessRows: 200);
var predictionResult = bestModel.Transform(testDf);
var survived = predictionResult.GetColumn<bool>("PredictedLabel");
var passengerId = predictionResult.GetColumn<float>("PassengerId");

var submissionDf = new DataFrame();
submissionDf["PassengerId"] = DataFrameColumn.Create("PassengerId", passengerId);
submissionDf["Survived"] = DataFrameColumn.Create("Survived", survived.Select(x => x ? 1 : 0));
DataFrame.WriteCsv(submissionDf, submissionCsvPath);

submissionDf.Head(10)

## How can I improve the result
If you are looking for improve the submission result, feature engineering should always be the first thing you try. The featurizer pipeline this notebook uses is automatically generated by [ML.Net ModelBuilder]() based on hard-code rules so there's a huge room for improvement! Bring your human knowledge and imagination when featurizing columns, digging more information behind them. For example. departure and arrival information is available from `Ticket`. `Name` also includes ton of useful information as well, last name might indicate their social status, which proves to be strongly related to survival rate. There's no limitation in feature engineering, use your imagination and explore!